<a href="https://colab.research.google.com/github/Jinzhao-Yu/BioStat615/blob/main/BIOSTAT615_Lecture_16_Fall_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BIOSTAT615 Lecture 16 - R

## 1. Fibonacci Numbers

In [1]:
#' A recusive implementation for fibonacci numbers
fib.v1 = function(n) {
  if ( n < 2 ) {
    return (n)
  } else {
    return (fib.v1(n-1) + fib.v1(n-2))
  }
}

In [2]:
system.time(print(fib.v1(30)))

[1] 832040


   user  system elapsed 
  1.620   0.016   2.207 

In [3]:
system.time(print(fib.v1(35)))

[1] 9227465


   user  system elapsed 
 16.199   0.087  18.217 

## 2. A non-redundant Fibonacci

In [4]:
#' A non-recusive implementation for fibonacci numbers
fib.v2 = function(n) {
  fibs = integer(length=n)
  fibs[1] = 1
  fibs[2] = 1
  for(i in seq(3,n,1)) {
    fibs[i] = fibs[i-1] + fibs[i-2]
  }
  return(fibs[n])
}

In [5]:
system.time(print(fib.v2(30)))

[1] 832040


   user  system elapsed 
  0.006   0.000   0.006 

In [6]:
system.time(print(fib.v2(35)))

[1] 9227465


   user  system elapsed 
  0.001   0.000   0.001 

## 3. Manhattan-Tourist Problem

In [7]:
#' Manhattan-Tourist Problem - dynamic programming
#' @param hcost : n * (m-1) matrix for horizontal move cost
#' @param vcost : (n-1) * m matrix for vertical move cost
#' @return optimal cost to (row,col) 
MTP.v1 = function(hcost, vcost) {
  n = nrow(hcost)
  m = ncol(vcost)
  stopifnot(m - 1 == ncol(hcost))
  stopifnot(n - 1 == nrow(vcost))
  optCost = matrix(NA, n, m)
  optMove = matrix(-1, n, m)
  for(i in 1:n) {
    for(j in 1:m) {
      if ( i == 1 ) {
        if ( j == 1 ) {     ## i == 1, j == 1
          optCost[i,j] = 0
        } else {            ## i == 1, j > 1
          optCost[i,j] = optCost[i,j-1] + hcost[i,j-1]
          optMove[i,j] = 0
        }
      } else if ( j == 1 ) { ## i > 1, j == 1
          optCost[i,j] = optCost[i-1,j] + vcost[i-1,j]
          optMove[i,j] = 1
      } else {
          h = optCost[i,j-1] + hcost[i,j-1]
          v = optCost[i-1,j] + vcost[i-1,j]
          if ( h > v ) {
            optMove[i,j] <- 1 ## optimal move is vertical
            optCost[i,j] <- v
          } else {
            optMove[i,j] <- 0 ## optimal move is horizontal
            optCost[i,j] <- h
          }
      }
    }
  }
  return( list(cost=optCost,move=optMove) )
}

In [8]:
hcost = matrix(c(4,7,6,1,1,2,4,8,6,5,0,5,1,4,8,7,9,0,7,5),5,4)
vcost = matrix(c(0,9,1,3,6,7,8,6,6,1,4,6,2,0,8,0,4,6,9,7),4,5)
print(hcost)
print(vcost)

     [,1] [,2] [,3] [,4]
[1,]    4    2    0    7
[2,]    7    4    5    9
[3,]    6    8    1    0
[4,]    1    6    4    7
[5,]    1    5    8    5
     [,1] [,2] [,3] [,4] [,5]
[1,]    0    6    6    2    4
[2,]    9    7    1    0    6
[3,]    1    8    4    8    9
[4,]    3    6    6    0    7


In [9]:
print(rst <- MTP.v1(hcost, vcost))

$cost
     [,1] [,2] [,3] [,4] [,5]
[1,]    0    4    6    6   13
[2,]    0    7   11    8   17
[3,]    9   14   12    8    8
[4,]   10   11   16   16   17
[5,]   13   14   19   16   21

$move
     [,1] [,2] [,3] [,4] [,5]
[1,]   -1    0    0    0    0
[2,]    1    0    0    1    0
[3,]    1    1    1    1    0
[4,]    1    0    1    1    1
[5,]    1    0    0    1    0



## 4. Backtracking in the Manhattan Tourist Problem

In [10]:
#' MTP.path() - backtrack optimal patch
#' @param cost - optimal cost output of MTP
#' @param move - optimal move output of MTP
#' @return A dataframe with (row, col, cost) in each step
MTP.path = function(cost, move) {
  m = nrow(move)
  n = ncol(move)
  k = m + n - 1 ## total number of moves needed
  df = data.frame(row=rep(NA,k),col=rep(NA,k),cost=rep(NA,k))
  x = m; y = n
  for(i in seq(k,1,-1)) { # backtrack from destination
    df[i,] = c(x,y,cost[x,y])
    if ( move[x,y] == 1 ) { # vertical move
      x = x-1
    } else {  # horizontal move
      y = y-1
    }
  }
  return(df)
}

In [11]:
print(MTP.path(rst$cost,rst$move))

  row col cost
1   1   1    0
2   1   2    4
3   1   3    6
4   1   4    6
5   2   4    8
6   3   4    8
7   4   4   16
8   5   4   16
9   5   5   21


## 5. Edit Distance Problem

In [12]:
#' edit.distance() - Align two strings
#' @param s1, s2 - two strings to align
#' @param gapcost - penalty for insertion/deletion
#' @param subcost - penalty for substitution
edit.distance = function(s1, s2, gapcost=1, subcost=1) {
  x = c("",strsplit(s1,"")[[1]])  ## split by chracter after an empty string
  y = c("",strsplit(s2,"")[[1]])
  n = length(x); m = length(y)    ## n, m are the length of strings
  cost = matrix(NA,n,m)           ## penalty for optimal path
  move = matrix(NA,n,m)     ## -1:deletion, 0:match/substitution 1:insertion
  for(i in 1:n) {
    for(j in 1:m) {
      if ( i == 1 ) {
        if ( j == 1 ) { ## i=1,j=1
          cost[i,j] = 0
        } else {        ## i=1,j>1
          cost[i,j] = cost[i,j-1] + gapcost
          move[i,j] = 1 ## insertion
        }
      } else if ( j == 1 ) {  ## i>1,j==1
        cost[i,j] = cost[i-1,j] + gapcost
        move[i,j] = -1 ## deletion
      } else {    ## i>1,j>1
        icost = cost[i,j-1] + gapcost ## cost for insertion
        dcost = cost[i-1,j] + gapcost ## cost for deletion
        scost = cost[i-1,j-1] + (x[i] != y[j]) * subcost ## cost for match/subst
        cost[i,j] = min(c(dcost,scost,icost)) ## pick the minimum cost
        move[i,j] = which.min(c(dcost, scost, icost))-2 ## pick minimum path
      }
    }
  }
  ## construct alignment
  i = n; j = m;   ## indices to backtrack
  ax = c(); ay = c(); edit = c()  ## aligned strings
  while((i > 1) || (j > 1)) {
    if ( move[i,j] == 0 ) { ## match or substitution
      ax = c(x[i],ax) 
      ay = c(y[j],ay)
      edit = c( ifelse(x[i]==y[j],".","S"), edit)
      i = i-1
      j = j-1
    } else if ( move[i,j] > 0 ) { ## insertion
      ax = c("-",ax)
      ay = c(y[j],ay)
      j = j-1
      edit = c("I", edit)
    } else {             ## deletion
      ax = c(x[i],ax)
      ay = c("-",ay)
      i = i-1
      edit = c("D", edit)
    }
  }
  return(list(cost=cost,move=move,alignd.x=paste0(ax,collapse=""),
              aligned.y=paste0(ay,collapse=""),edit.op=paste0(edit,collapse="")))
}

In [13]:
print(edit.distance("FOOD","MONEY"))

$cost
     [,1] [,2] [,3] [,4] [,5] [,6]
[1,]    0    1    2    3    4    5
[2,]    1    1    2    3    4    5
[3,]    2    2    1    2    3    4
[4,]    3    3    2    2    3    4
[5,]    4    4    3    3    3    4

$move
     [,1] [,2] [,3] [,4] [,5] [,6]
[1,]   NA    1    1    1    1    1
[2,]   -1    0    0    0    0    0
[3,]   -1   -1    0    1    1    1
[4,]   -1   -1   -1    0    0    0
[5,]   -1   -1   -1   -1    0    0

$alignd.x
[1] "FO-OD"

$aligned.y
[1] "MONEY"

$edit.op
[1] "S.ISS"



In [14]:
print(edit.distance("ALTRUISTIC","ALGORITHM"))

$cost
      [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10]
 [1,]    0    1    2    3    4    5    6    7    8     9
 [2,]    1    0    1    2    3    4    5    6    7     8
 [3,]    2    1    0    1    2    3    4    5    6     7
 [4,]    3    2    1    1    2    3    4    4    5     6
 [5,]    4    3    2    2    2    2    3    4    5     6
 [6,]    5    4    3    3    3    3    3    4    5     6
 [7,]    6    5    4    4    4    4    3    4    5     6
 [8,]    7    6    5    5    5    5    4    4    5     6
 [9,]    8    7    6    6    6    6    5    4    5     6
[10,]    9    8    7    7    7    7    6    5    5     6
[11,]   10    9    8    8    8    8    7    6    6     6

$move
      [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10]
 [1,]   NA    1    1    1    1    1    1    1    1     1
 [2,]   -1    0    1    1    1    1    1    1    1     1
 [3,]   -1   -1    0    1    1    1    1    1    1     1
 [4,]   -1   -1   -1    0    0    0    0    0    1     1
 [5,]   -1   -1   

In [15]:
print(edit.distance("AGTGTCGG","GTCGGA"))

$cost
      [,1] [,2] [,3] [,4] [,5] [,6] [,7]
 [1,]    0    1    2    3    4    5    6
 [2,]    1    1    2    3    4    5    5
 [3,]    2    1    2    3    3    4    5
 [4,]    3    2    1    2    3    4    5
 [5,]    4    3    2    2    2    3    4
 [6,]    5    4    3    3    3    3    4
 [7,]    6    5    4    3    4    4    4
 [8,]    7    6    5    4    3    4    5
 [9,]    8    7    6    5    4    3    4

$move
      [,1] [,2] [,3] [,4] [,5] [,6] [,7]
 [1,]   NA    1    1    1    1    1    1
 [2,]   -1    0    0    0    0    0    0
 [3,]   -1    0    0    0    0    0    1
 [4,]   -1   -1    0    1    1    0    0
 [5,]   -1   -1   -1    0    0    0    1
 [6,]   -1   -1   -1   -1   -1    0    0
 [7,]   -1   -1   -1    0   -1   -1    0
 [8,]   -1   -1   -1   -1    0    0   -1
 [9,]   -1   -1   -1   -1   -1    0    1

$alignd.x
[1] "AGTGTCGG-"

$aligned.y
[1] "-GT--CGGA"

$edit.op
[1] "D..DD...I"

